# Overview

The project use openCV and ultralytics YOLO11m model with BoT-SORT tracker to keep track of the `time` that people spend in store

1. Imports
2. capture
3. while True if ret,cap.read
4. model.track persist
5. Once you got the id, start thier timer, use dict
5. if results.class_id=people
6. put small bbox above on bbox of time
7. the time in box, current-(the object first seen)
7. if q pressed,exit

# Imports

In [18]:
import cv2
from ultralytics import YOLO,settings
settings.update({'mlflow':True})
settings.reset()

In [19]:
import mlflow
import torch

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model=YOLO('yolo11m.pt').to(device)

In [20]:
# for result in results:
#     print(result.boxes[0])
    
# ultralytics.engine.results.Boxes object with attributes:

# cls: tensor([0.])
# conf: tensor([0.8821])
# data: tensor([[352.4166, 111.2105, 500.6862, 487.0408,   1.0000,   0.8821,   0.0000]])
# id: tensor([1.])
# is_track: True
# orig_shape: (720, 1270)
# shape: torch.Size([1, 7])
# xywh: tensor([[426.5514, 299.1257, 148.2697, 375.8303]])
# xywhn: tensor([[0.3359, 0.4155, 0.1167, 0.5220]])
# xyxy: tensor([[352.4166, 111.2105, 500.6862, 487.0408]])
# xyxyn: tensor([[0.2775, 0.1545, 0.3942, 0.6764]])

In [21]:
model.names

{0: 'person',
 1: 'bicycle',
 2: 'car',
 3: 'motorcycle',
 4: 'airplane',
 5: 'bus',
 6: 'train',
 7: 'truck',
 8: 'boat',
 9: 'traffic light',
 10: 'fire hydrant',
 11: 'stop sign',
 12: 'parking meter',
 13: 'bench',
 14: 'bird',
 15: 'cat',
 16: 'dog',
 17: 'horse',
 18: 'sheep',
 19: 'cow',
 20: 'elephant',
 21: 'bear',
 22: 'zebra',
 23: 'giraffe',
 24: 'backpack',
 25: 'umbrella',
 26: 'handbag',
 27: 'tie',
 28: 'suitcase',
 29: 'frisbee',
 30: 'skis',
 31: 'snowboard',
 32: 'sports ball',
 33: 'kite',
 34: 'baseball bat',
 35: 'baseball glove',
 36: 'skateboard',
 37: 'surfboard',
 38: 'tennis racket',
 39: 'bottle',
 40: 'wine glass',
 41: 'cup',
 42: 'fork',
 43: 'knife',
 44: 'spoon',
 45: 'bowl',
 46: 'banana',
 47: 'apple',
 48: 'sandwich',
 49: 'orange',
 50: 'broccoli',
 51: 'carrot',
 52: 'hot dog',
 53: 'pizza',
 54: 'donut',
 55: 'cake',
 56: 'chair',
 57: 'couch',
 58: 'potted plant',
 59: 'bed',
 60: 'dining table',
 61: 'toilet',
 62: 'tv',
 63: 'laptop',
 64: 'mou

In [22]:
from time import perf_counter

In [23]:
!pip install easydict

In [24]:
import easydict
from deep_sort_pytorch.utils.parser import get_config
from deep_sort_pytorch.deep_sort import DeepSort

deepsort=None
def init_tracker():
    global deepsort
    cfg_deep = get_config()
    cfg_deep.merge_from_file("deep_sort_pytorch/configs/deep_sort.yaml")

    deepsort= DeepSort(cfg_deep.DEEPSORT.REID_CKPT,
                            max_dist=cfg_deep.DEEPSORT.MAX_DIST, min_confidence=cfg_deep.DEEPSORT.MIN_CONFIDENCE,
                            nms_max_overlap=cfg_deep.DEEPSORT.NMS_MAX_OVERLAP, max_iou_distance=cfg_deep.DEEPSORT.MAX_IOU_DISTANCE,
                            max_age=cfg_deep.DEEPSORT.MAX_AGE, n_init=cfg_deep.DEEPSORT.N_INIT, nn_budget=cfg_deep.DEEPSORT.NN_BUDGET,
                            use_cuda=True)
init_tracker()

In [ ]:
import numpy as np

cap = cv2.VideoCapture('streams/store.mp4')
# Get frame size
ret, frame = cap.read()
if not ret:
    print("Error: Could not read frame.")
    exit()

frame_height, frame_width = frame.shape[:2]
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Use 'mp4v' or 'X264' for .mp4 files
out_video = cv2.VideoWriter('output_video.mp4', fourcc, 20.0, (frame_width, frame_height))
# frame_width = int(cap.get(3)) 
# frame_height = int(cap.get(4)) 
   
# size = (frame_width, frame_height) 
# fourcc = cv2.VideoWriter_fourcc(*'MJPG')  # Try different codecs like 'MJPG', 'X264', etc.
# out_video = cv2.VideoWriter('output_video.avi',  
#                          fourcc, 
#                          10, 
#                          size) 
tracker_time = {}

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    results = model.predict(frame, classes=[0])
    
    for r in results:
        if len(r.boxes)>0:
            bbox_xywh=r.boxes.xywh.cpu().numpy().astype('float64')
            confs=r.boxes.conf.cpu().numpy().astype('float64')
            clss=r.boxes.cls.cpu().numpy().astype('float64')
            
            output=deepsort.update(bbox_xywh,confs,clss,frame)
            
            if len(output) > 0:
                for out in output:
                    x1,y1,x2,y2,track_id=map(int,out[:5])
                    # time
                    if track_id not in tracker_time:
                        tracker_time[track_id] = [perf_counter(), perf_counter()]
                    tracker_time[track_id][1] = perf_counter()
                    
                    
                    time_in_store = tracker_time[track_id][1] - tracker_time[track_id][0]
                    
                    
                    cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                    cv2.putText(frame, f"ID:{track_id} Time:{time_in_store:.2f}s", 
                              (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)
    
    cv2.imshow('frame', frame)
    # out_video.write(frame)
    if cv2.waitKey(1) & 0xFF == ord('q'): 
        break

cap.release()
out_video.release()
cv2.destroyAllWindows()


0: 384x640 13 persons, 10.4ms
Speed: 1.8ms preprocess, 10.4ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 13 persons, 9.7ms
Speed: 1.1ms preprocess, 9.7ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 13 persons, 10.1ms
Speed: 1.2ms preprocess, 10.1ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 13 persons, 10.2ms
Speed: 2.2ms preprocess, 10.2ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 13 persons, 9.8ms
Speed: 1.3ms preprocess, 9.8ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 13 persons, 9.7ms
Speed: 1.4ms preprocess, 9.7ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 13 persons, 9.7ms
Speed: 1.4ms preprocess, 9.7ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 13 persons, 9.8ms
Speed: 1.4ms preprocess, 9.8ms inference, 0.8ms postprocess per image at shape (1,